In [1]:
# Define the project directory inorder to complie with the reproduceability of the code
project_path = """/home/dhamaks/GitHub/Daphnet-Freezing-of-Gait-Forecasting/"""

In [2]:
# Import all the required libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

2023-03-31 15:46:36.500384: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-31 15:46:36.537279: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 15:46:37.026484: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_csv(project_path + "Daphnet-Freezing-of-Gait-Dataset/consolidated_dataset.csv", sep='\t', header=0)
df

,Time in millisecond,Ankle_acc - horizontal forward,Ankle_acc - vertical,Ankle_acc - horizontal lateral,Upper_leg_acc - horizontal forward,Upper_leg_acc - vertical,Upper_leg_acc - horizontal lateral,Trunk_acc - horizontal forward,Trunk_acc - vertical,Trunk_acc - horizontal lateral,Annotation,File S,File R,File
0,15,70,39,-970,0,0,0,0,0,0,0,1.0,1.0,S01R01
1,31,70,39,-970,0,0,0,0,0,0,0,1.0,1.0,S01R01
2,46,60,49,-960,0,0,0,0,0,0,0,1.0,1.0,S01R01
3,62,60,49,-960,0,0,0,0,0,0,0,1.0,1.0,S01R01
4,78,50,39,-960,0,0,0,0,0,0,0,1.0,1.0,S01R01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917882,3020296,-131,107,-960,0,0,0,0,0,0,0,10.0,1.0,S10R01
1917883,3020312,-121,127,-970,0,0,0,0,0,0,0,10.0,1.0,S10R01
1917884,3020328,-141,117,-960,0,0,0,0,0,0,0,10.0,1.0,S10R01
1917885,3020343,-131,127,-980,0,0,0,0,0,0,0,10.0,1.0,S10R01


In [4]:
def split_sequences_single_output(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return train_test_split(np.array(X), np.array(y), test_size=0.2, shuffle=False)

def split_sequences_multi_output(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences)-1:
            break
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return train_test_split(np.array(X), np.array(y), test_size=0.2, shuffle=False)


# Single-Output-stacked-LSTM

In [5]:
n_steps = 1

X_train, X_test, y_train, y_test = split_sequences_single_output(df.drop(['Time in millisecond', 
                                                                            'File S', 
                                                                            'File R', 
                                                                            'File'], 
                                                                            axis=1).values, 
                                                                n_steps)

In [6]:
model = tf.keras.models.load_model(project_path + f"models/single-output-stacked-LSTM/n_steps_{n_steps}/S00R00.tf")

2023-03-31 15:46:40.239127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-31 15:46:40.256234: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:
X_train.shape

(1534309, 1, 9)

In [11]:
model.predict(X_test)

11987/11987 [==============================] - 20s 2ms/step


array([[0.33333337, 0.3333333 , 0.33333328],
       [0.33333337, 0.3333333 , 0.33333328],
       [0.33333337, 0.3333333 , 0.33333328],
       ...,
       [0.33333337, 0.3333333 , 0.33333328],
       [0.33333337, 0.3333333 , 0.33333328],
       [0.33332992, 0.33333623, 0.3333338 ]], dtype=float32)

In [12]:
model.predict(X_train)

47948/47948 [==============================] - 80s 2ms/step


array([[0.33333337, 0.3333333 , 0.33333328],
       [0.33333337, 0.3333333 , 0.33333328],
       [0.33333337, 0.3333333 , 0.33333328],
       ...,
       [0.33333337, 0.3333333 , 0.33333328],
       [0.33333337, 0.3333333 , 0.33333328],
       [0.33333337, 0.3333333 , 0.33333328]], dtype=float32)

In [16]:
X_train[0]

array([[  70,   39, -970,    0,    0,    0,    0,    0,    0]])

In [ ]:
test_case_X = 